In [1]:
import torch
from torch_geometric.data import InMemoryDataset, download_url, Data, Batch
from torch import nn
from torch import functional as F
import os
import pandas as pd
import numpy as np
import pickle
import itertools
import jax
from jax import numpy as jnp
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
import mendeleev

In [2]:
from mendeleev import element

In [3]:
C, H, N, O, F = element(["C", "H", "N", "O", "F"])

In [4]:
rows = []
for e in [C, H, N, O, F]:
    row = [e.atomic_radius,
           e.atomic_volume,
           e.atomic_weight,
           e.boiling_point,
           e.covalent_radius_bragg,
           e.dipole_polarizability,
           e.electron_affinity,
           e.en_ghosh,
           e.vdw_radius]
    rows.append(row)

In [5]:
properties=np.array(rows)

In [6]:
properties[properties == None] = 0

In [7]:
emms = MinMaxScaler([-5, 5])
props = properties.astype(np.float32)
props = emms.fit_transform(props)

In [8]:
def get_metaedge_features(incident_edges, coords):
    head2tail = []
    triads_ht = []
    head2head = []
    triads_hh = []
    tail2tail = []
    triads_tt = []
    for x, i in enumerate(incident_edges):
        for y, j in enumerate(incident_edges):
            if i[1] == j[0]:
                head2tail.append((x, y))
                triads_ht.append((i[0], i[1], j[1]))
            if i[0] == j[0] and x != y:
                tail2tail.append((x, y))
                triads_hh.append((i[1], i[0], j[1]))
            if i[1] == j[1] and x != y:
                head2head.append((x, y))
                triads_tt.append((i[0], i[1], j[0]))

    head2tail = np.array(head2tail)
    head2head = np.array(head2head)
    tail2tail = np.array(tail2tail)
    all_triads = triads_ht + triads_hh + triads_tt
    non_empty = [array for array in [head2tail, head2head, tail2tail] if len(array) > 0]
    metaedges = np.concatenate(non_empty)
    angles = []
    for triad in all_triads:
        angles.append(get_angle(coords, triad))
    return metaedges, np.array(angles)

def get_angle(array, triad):
    """
    Array of distances to angles between edges
    """
    i, j, k = triad
    a =  array[i] - array[j]
    b =  array[k] - array[j]
    a = a/np.linalg.norm(a)
    b = b/np.linalg.norm(b)
    return a@b

In [9]:
def get_edge_features(coords, dipole_moment, eps = 0.000001):
    norm_dipole = np.linalg.norm(dipole_moment) # get the norm to normalize the vector and find angles
    distmat = squareform(pdist(coords)) # get dist_mat to find edges
    np.fill_diagonal(distmat, np.nan) # fill to avoid ranking problem
    rankings = distmat.argsort(axis=0) # order distance matrix to get n-neighborhood of each edge
    G = nx.from_numpy_matrix(distmat, create_using=nx.DiGraph())
    G.remove_edges_from(nx.selfloop_edges(G)) # remove to avoid self edges
    edgelist = nx.to_edgelist(G)
    edgelist, edge_features = edge_list_to_numpy(edgelist) #get edges and distances
    G = nx.from_numpy_matrix(rankings+1, create_using=nx.DiGraph())
    G.remove_edges_from(nx.selfloop_edges(G))
    edgelist2 = nx.to_edgelist(G)
    edgelist2, edge_rankings = edge_list_to_numpy(edgelist2) # get rankings
    edge_rankings = edge_rankings - 1
    n_edges = edgelist.shape[0]
    coords_nodes = coords[edgelist] #select nodes in edges
    vectors_edges = coords_nodes.transpose(0, 2, 1)[:, :, 1] - coords_nodes.transpose(0, 2, 1)[:, :, 0] # get vector of each edge
    vectors_edges_normalized = vectors_edges/(np.linalg.norm(vectors_edges, axis=1) + eps)[:,None] 
    dipole_moment_normalized = dipole_moment/(norm_dipole + eps)
    angles_dipole_moment = np.dot(vectors_edges_normalized, dipole_moment) # do dot product to find angles
    ranks = np.zeros([n_edges, 6])
    edge_rankings[edge_rankings > 4] = 5 # replace to impose same dimensionality in all molecules
    ranks[np.arange(n_edges), edge_rankings] = 1
    edge_features = np.concatenate([edge_features[:,None], ranks, angles_dipole_moment[:,None]], axis=1) # concatenate all features
    return edgelist, edge_features

In [10]:
def edge_list_to_numpy(edgelist):
    tail = []
    head = []
    weight = []
    for edge in list(edgelist):
        tail.append(edge[0])
        head.append(edge[1])
        weight.append(edge[2]["weight"])
    tail = np.array(tail)[:,None]
    head = np.array(head)[:,None]
    weight = np.array(weight)
    return np.concatenate([tail, head], axis=1), weight


In [11]:
target = pd.read_csv("raw/train.csv")

In [12]:
edges = target["type"].unique()

In [13]:
edge_to_int = {edges[i]:i for i in range(len(edges))}

In [14]:
class SCDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, training= True):
        super().__init__(root, transform, pre_transform)
        self.filenames = pd.read_csv("raw/processed_names.csv")
        self.charges = pd.read_csv("raw/mulliken_charges.csv")
        self.magnetic_shieldings = pd.read_csv("raw/magnetic_shielding_tensors.csv")
        self.dipole_moments = pd.read_csv("raw/dipole_moments.csv")
        self.potential_energy = pd.read_csv("raw/potential_energy.csv")
        self.target = pd.read_csv("raw/train.csv")
        self.structures = pd.read_csv("raw/structures.csv")
        self.molecule_names = molecule_names = np.unique(self.potential_energy["molecule_name"])
        if training:
            self.training_mask = np.loadtxt("./raw/training_mask2.csv").astype(bool)
            self.molecule_names = self.molecule_names[self.training_mask]
            
    def len(self) -> int:
        return len(self.molecule_names)
    def standarize(self):
        mms = MinMaxScaler([-4, 4])
        self.charges["mulliken_charge"] = mms.fit_transform(self.charges[["mulliken_charge"]]).squeeze()
        self.magnetic_shieldings.iloc[:, 2:] = mms.fit_transform(self.magnetic_shieldings.iloc[:, 2:])
        self.dipole_moments.iloc[:, 1:] = mms.fit_transform(self.dipole_moments.iloc[:, 1:])
        self.potential_energy["potential_energy"] = mms.fit_transform(self.potential_energy[["potential_energy"]]).squeeze()

    def preprocess(self, k = None):
        charges = self.charges
        magnetic_shieldings = self.magnetic_shieldings
        dipole_moments = self.dipole_moments
        potential_energy = self.potential_energy
        molecule_names = self.molecule_names
        target = self.target
        structures = self.structures
        dfs = [charges, magnetic_shieldings, dipole_moments, potential_energy, target, structures]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].set_index("molecule_name", drop=True)
        charges, magnetic_shieldings, dipole_moments, potential_energy, target, structures = dfs
        atoms = structures["atom"].unique()
        atoms_id = {atoms[i]:i for i in range(len(atoms))}
        training_mask = []
        for x, name in enumerate(list(molecule_names)):
            any_training_edges = True
            coords = structures.loc[name][["x", "y", "z"]].to_numpy()
            n_nodes = coords.shape[0]
            print("{}/{}".format(x + 1, len(molecule_names)), end = "\r")
            # adj_mat
            with open("./processed/{}_adj_mat.csv".format(name), "wb") as f:
                np.savetxt(f, get_distance_matrix(coords, k))
            atom_types = structures.loc[name]["atom"].replace(atoms_id).to_numpy()
            atom_onehot = np.zeros([n_nodes, len(atoms)])
            atom_onehot[np.arange(0, n_nodes), atom_types] = 1
            charge = charges.loc[name]["mulliken_charge"].to_numpy()
            shieldings = magnetic_shieldings.loc[name].iloc[:, 2:].to_numpy()
            node_features = np.concatenate([charge[:, None], shieldings, atom_onehot], axis=1)
            with open("./processed/{}_node_attr.csv".format(name), "wb") as f:
                np.savetxt(f, node_features)
            try:
                edges_target = target.loc[[name]]
                training_mask.append(True)
            except KeyError:
                training_mask.append(False)
                any_training_edges = False
                
            if any_training_edges:
                edges_target["type"] = edges_target["type"].replace(edge_to_int).astype(np.int64)
                scalar_coupling = edges_target.loc[:, ["atom_index_0", "atom_index_1","type","scalar_coupling_constant"]].to_numpy()
            else:
                scalar_coupling = np.array([-1, -1, -1, 0])
            with open("./processed/{}_target.csv".format(name), "wb") as f:
                np.savetxt(f, scalar_coupling)
            # Graph features
            dipole_moment = dipole_moments.loc[name]
            norm_dipole = np.array([np.linalg.norm(dipole_moment)])
            potential = potential_energy.loc[name]
            graph_features = (np.concatenate([dipole_moment, norm_dipole, potential, np.array([n_nodes])]))
            with open("./processed/{}_graph_feautures.csv".format(name), "wb") as f:
                np.savetxt(f, graph_features)
            # edge_features
            edgelist, edgeattr = get_edge_features(coords, dipole_moment)
            with open("./processed/{}_edge_list.csv".format(name), "wb") as f:
                np.savetxt(f, edgelist)
            with open("./processed/{}_edge_attr.csv".format(name), "wb") as f:
                np.savetxt(f, edgeattr)
        with open("./raw/training_mask2.csv".format(name), "wb") as f:
                np.savetxt(f, np.array(training_mask))
                
    def mem_load(self):
        self.mem = {}
        for i, molecule in enumerate(self.molecule_names):
            graph_features = pd.read_csv("./processed/{}_graph_feautures.csv".format(molecule), sep=" ", header=None).to_numpy()
            node_features = pd.read_csv("./processed/{}_node_attr.csv".format(molecule), sep=" ", header=None).to_numpy()
            atomtypes = node_features[:,-5:].argmax(axis=1)
            prop_atoms = props[atomtypes,:]
            n_nodes = node_features.shape[0]
            graph_features = np.tile(graph_features, [1, n_nodes]).T
            node_features = np.concatenate([prop_atoms, node_features, graph_features], axis=1)
            target =  pd.read_csv("./processed/{}_target.csv".format(molecule), sep=" ", header=None).to_numpy()
            edge_type = target[:,2]
            edge_type = np.concatenate([edge_type, edge_type], axis=0)
            edges_target = target[:,0:2]
            target = target[:,3]
            target = np.concatenate([target, target])
            edges_target = np.concatenate([edges_target, edges_target[:,::-1]], axis=0)
            edge_list = pd.read_csv("./processed/{}_edge_list.csv".format(molecule), sep=" ", header=None).to_numpy()
            edge_attr = pd.read_csv("./processed/{}_edge_attr.csv".format(molecule), sep=" ", header=None).to_numpy()
            metaedge_list = pd.read_csv("./processed/{}_metaedge_list.csv".format(molecule), sep=" ", header=None).to_numpy()
            metaedge_attr = pd.read_csv("./processed/{}_metaedge_attr.csv".format(molecule), sep=" ", header=None).to_numpy()
            data = Data(x=torch.Tensor(node_features), edge_index = torch.Tensor(edge_list).T, y=torch.Tensor(target), edge_attr = torch.Tensor(edge_attr))
            data.nodes_target = torch.Tensor(edges_target)
            data.nodes = n_nodes
            data.edges = edge_list.shape[0]
            data.types = torch.Tensor(edge_type)
            data.metaedge_list = metaedge_list
            data.metaedge_list = metaedge_attr
            # data.edge_cross = edgelist
            # data.nodes = node_features.shape[0]
            self.mem[molecule] = data
            print("{}/{}".format(i, len(self.molecule_names)), end = "\r")
            
    def __getitem__(self, idx):
        molecule = self.molecule_names[idx]
        return self.mem[molecule]
        

def get_distance_matrix(X, k=None):
    dist = squareform(pdist(X))
    if k is not None:
        non_k = dist.argsort(axis=1)[:, k+1:]
        dist[np.arange(0, dist.shape[0])[:,None], non_k] = 0
    return dist

def to_batch(list_graphs):
    n_nodes = 0
    for graph in list_graphs:
        graph["nodes_target"] += n_nodes
        n_nodes += graph.nodes
    return Batch.from_data_list(list_graphs) 

In [15]:
with open("./train_dataloader.pkl", "rb") as f:
    train_dataloader = pickle.load(f)
with open("./test_dataloader.pkl", "rb") as f:
    test_dataloader = pickle.load(f)

In [16]:
from torch_geometric.nn import GCNConv, GATv2Conv, GATConv, SAGEConv
import torch.nn as nn
import torch.nn.functional as F

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


In [17]:
class GATv2EncoderGated(nn.Module):
    def __init__(self, num_node_features, hidden_features, heads, n_layers, p_dropout):
        super().__init__()
        self.p_dropout = p_dropout
        assert n_layers > 1
        self.init_conv = GATv2Conv(num_node_features, hidden_features, heads=n_heads, dropout=p_dropout,  edge_dim=8, concat=False)
        self.layers = nn.ModuleList([GATv2Conv(hidden_features, hidden_features, heads=n_heads, dropout=p_dropout,  edge_dim=8, concat=False) for i in range(n_layers)])
        self.gates = nn.Parameter(torch.Tensor(n_layers))
        self.init_conv.apply(init_weights)
        for conv in self.layers:
            conv.apply(init_weights)
    def forward(self, x, edge_index, edge_attr):
        range_gates = torch.sigmoid(self.gates)
        x = self.init_conv(x, edge_index, edge_attr)
        for i, layer in enumerate(self.layers):
            x = F.leaky_relu(x)
            x = (range_gates[i])*layer(x, edge_index, edge_attr) + (1-range_gates[i])*x
        return x
    
class ResNetGated(nn.Module):
    def __init__(self, init_dim, hidden_dim, layers, p_dropout):
        super().__init__()
        self.p_dropout = p_dropout
        assert n_layers > 1
        self.layers = nn.ModuleList([nn.Sequential(nn.Linear(init_dim, hidden_dim),
                             nn.ReLU(),
                             nn.Dropout(p=p_dropout),
                             nn.Linear( hidden_dim, init_dim)) for i in range(layers)])
        self.gates = nn.Parameter(torch.Tensor(n_layers))
        self.layers.apply(init_weights)
    def forward(self, x):
        range_gates = torch.sigmoid(self.gates)
        for i, layer in enumerate(self.layers):
            x = F.relu(x)
            x = (range_gates[i])*layer(x) + (1-range_gates[i])*x
        return x

    
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, out_features, n_heads, n_layers, n_res, p_dropout):
        super().__init__()
        self.conv = GATv2EncoderGated(num_node_features, out_features, heads=n_heads, p_dropout=p_dropout,  n_layers=3)
        self.fcs = nn.ModuleList([nn.Sequential(ResNetGated(out_features*2, out_features*64, n_res, p_dropout),
                               nn.Linear(2 * out_features, 1)) for i in range(8)])
        for fc in self.fcs:
            fc.apply(init_weights)
    def forward(self, x, edge_index, edge_attr, edge_cross, types, return_embeddings = False):
        x = self.conv(x, edge_index, edge_attr)
        if return_embeddings:
            embeddings = x
        x = x[edge_cross]
        shp = x.shape
        x = x.transpose(1, 2).reshape([shp[0], shp[2]*2])
        xs = []
        for i in range(8):
            xs.append(self.fcs[i](x[types == i]))
        x = torch.concat(xs, axis=0)
        if return_embeddings:
            return x, embeddings
        return x

In [18]:
### Define the loss function
loss_fn = nn.MSELoss

lr= 0.001
weight_decay = 0.000001
p_dropout = 0.001
conv_features = 128
n_heads = 6
n_layers = 3
n_res = 2
### Set the random seed for reproducible results
torch.manual_seed(0)

gcn = GCN(29, conv_features, n_heads, n_layers, n_res, p_dropout=p_dropout)
params_to_optimize = [
    {'params': gcn.parameters()}
]
optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=weight_decay)
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
gcn.to(device)

Selected device: cuda


GCN(
  (conv): GATv2EncoderGated(
    (init_conv): GATv2Conv(29, 128, heads=6)
    (layers): ModuleList(
      (0): GATv2Conv(128, 128, heads=6)
      (1): GATv2Conv(128, 128, heads=6)
      (2): GATv2Conv(128, 128, heads=6)
    )
  )
  (fcs): ModuleList(
    (0): Sequential(
      (0): ResNetGated(
        (layers): ModuleList(
          (0): Sequential(
            (0): Linear(in_features=256, out_features=8192, bias=True)
            (1): ReLU()
            (2): Dropout(p=0.001, inplace=False)
            (3): Linear(in_features=8192, out_features=256, bias=True)
          )
          (1): Sequential(
            (0): Linear(in_features=256, out_features=8192, bias=True)
            (1): ReLU()
            (2): Dropout(p=0.001, inplace=False)
            (3): Linear(in_features=8192, out_features=256, bias=True)
          )
        )
      )
      (1): Linear(in_features=256, out_features=1, bias=True)
    )
    (1): Sequential(
      (0): ResNetGated(
        (layers): ModuleList(


In [19]:
saved_model = "./saved_models/with_resnet_2022-02-02-10:50:52_lr=0.001_wd=1e-06_p=0.001_conv_features=128_n_layers=3_n_res=2.pth"
gcn.load_state_dict(torch.load(saved_model))

<All keys matched successfully>

In [20]:
import datetime
date = datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')

RUN = date + "_lr={}_wd={}_p={}_conv_features={}_n_layers={}_n_res={}".format(lr,
                                                            weight_decay,
                                                            p_dropout,
                                                            conv_features,
                                                             n_layers,
                                                            n_res
                                                            )
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./runs/{}".format(RUN))

In [21]:
def train(model, device, data ,loss_fn, optimizer, batch_acc):
    model.train()
    train_losses = []
    optimizer.zero_grad()
    for i, batch in enumerate(data):
        x, edge_index, edge_attr, target, edge_cross, types = (batch["x"],
                                                               batch["edge_index"],
                                                               batch["edge_attr"],
                                                               batch["y"],
                                                               batch["nodes_target"],
                                                               batch["types"])
        min_2 = types [types < 2].shape[0]
        max_2 = types [types < 3].shape[0]
        types_cpu = types.numpy()
        sort_index = torch.Tensor(types.numpy().argsort(kind="stable")).long()
        target = target[sort_index]
        x, edge_index, edge_attr, target, edge_cross, types = x.to(device), \
                                                            edge_index.long().to(device), \
                                                            edge_attr.to(device), \
                                                            target.to(device),\
                                                            edge_cross.long().to(device), \
                                                            types.long().to(device)
        logits = model(x, edge_index, edge_attr, edge_cross, types)
        loss = loss_fn()
        output=loss(logits.squeeze()[min_2:max_2], target.squeeze()[min_2:max_2])
        if batch_acc == 1:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
        elif (i > 0) and (i & batch_acc == 0):
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
        train_loss = output.data.cpu().numpy()
        train_losses.append(train_loss)
        
    return np.mean(train_losses)

### Testing function
def test(model, device, data, loss_fn):
    # Set evaluation mode for encoder and decoder
    model.eval()
    test_losses = []
    with torch.no_grad(): # No need to track the gradients
        for i, batch in enumerate(data):
            x, edge_index, edge_attr, target, edge_cross, types = (batch["x"],
                                                               batch["edge_index"],
                                                               batch["edge_attr"],
                                                               batch["y"],
                                                               batch["nodes_target"],
                                                               batch["types"])
            types_cpu = types.numpy()
            min_2 = types [types < 2].shape[0]
            max_2 = types [types < 3].shape[0]
            sort_index = torch.Tensor(types.numpy().argsort(kind="stable")).long()
            target = target[sort_index]
            x, edge_index, edge_attr, target, edge_cross, types = x.to(device), \
                                                                edge_index.long().to(device), \
                                                                edge_attr.to(device), \
                                                                target.to(device),\
                                                                edge_cross.long().to(device), \
                                                                types.long().to(device)
            logits = model(x, edge_index, edge_attr, edge_cross, types)
            loss = loss_fn()
            output=loss(logits.squeeze()[min_2:max_2], target.squeeze()[min_2:max_2])
            test_loss = output.data.cpu().numpy()
            test_losses.append(test_loss)
    return np.mean(test_losses)

In [22]:
best_loss = 1000
num_epochs = 100
diz_loss = {'train_loss':[],'val_loss':[]}
batch_acc = 1
for epoch in range(num_epochs):
    if epoch >= 25 and epoch % 25 == 0:
        batch_acc *= 2
        for param in optim.param_groups:
            param["weight_decay"] = param["weight_decay"]/2
    train_loss = train(gcn, device, train_dataloader, loss_fn, optim, batch_acc)
    test_loss = test(gcn, device, test_dataloader, loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t \t val loss {}'.format(epoch + 1, num_epochs, train_loss, test_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(test_loss)
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/test', test_loss, epoch)
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(gcn.state_dict(), "./saved_models/MAE_resnet_{}.pth".format(RUN))


 EPOCH 1/100 	 train loss 0.6375083923339844 	 	 val loss 0.7725716829299927

 EPOCH 2/100 	 train loss 0.6342358589172363 	 	 val loss 0.7696903347969055

 EPOCH 3/100 	 train loss 0.6401548981666565 	 	 val loss 0.7825977802276611

 EPOCH 4/100 	 train loss 0.6398143172264099 	 	 val loss 0.770659863948822

 EPOCH 5/100 	 train loss 0.6322264671325684 	 	 val loss 0.771438717842102

 EPOCH 6/100 	 train loss 0.6369910836219788 	 	 val loss 0.7668255567550659

 EPOCH 7/100 	 train loss 0.6310602426528931 	 	 val loss 0.7726253271102905

 EPOCH 8/100 	 train loss 0.6388031840324402 	 	 val loss 0.7729706168174744

 EPOCH 9/100 	 train loss 0.637401282787323 	 	 val loss 0.7875005006790161

 EPOCH 10/100 	 train loss 0.6361809372901917 	 	 val loss 0.7678966522216797

 EPOCH 11/100 	 train loss 0.6414677500724792 	 	 val loss 0.7818551063537598

 EPOCH 12/100 	 train loss 0.6370216608047485 	 	 val loss 0.7646304368972778

 EPOCH 13/100 	 train loss 0.6388554573059082 	 	 val loss 0.77

In [23]:
print(RUN)

2022-02-08-09:32:26_lr=0.001_wd=1e-06_p=0.001_conv_features=128_n_layers=3_n_res=2
